# 필수과제1
- 데이터셋을 드릴 예정
- 이커머스 데이터를 통해 고객의 구매 패턴, 주기등을 파악을 해서 기준을 정한다!
- 그 기준에 따라 RFM에 대한 측정 기준별 월별 지표가 만들어 질 것
- RFM 지표를 가지고 -> 세그먼트를 만들 수 있다.
    - 방법론은 자유롭게 진행하시면 된다.
    - kmeans 및 다른 기타 클러스터링을 진행하시면 되는데 ->내 데이터의 분포에 따른 적절한 클러스터링 작업을 해야 한다.
- 레이블이 만들어 지면
    - 레이블은 202201 ~ 202301 데이터라고 하면, 3개월 기준이라 가정하면 202204 데이터가 확보가 된다.
    - 202204 - (202201~202203의 주문데이터)
    - 202205 - (202202~202204의 주문데이터)
    - 202206 .. 월별
- 월별의 레이블이 달라지게 만들어진다.
- 그 레이블의 기준으로 -> RFM 지표라 한다면 해당 지표가 어떤 식의 수렴하는 기준 값이 있는지 찾기
- 그 기준 값을 찾고 -> 해당기준 값으로 새로운 세그먼트를 만들어야 한다.
- 마지막 최종 비교는
    - 기존 세그먼트 ( 레이블을 만든 세그먼트 )
    - 내가 인사이트를 찾아서 만든 세그먼트
    - 둘 세그먼트의 차이를 비교분석 ( 기초통계, 카운팅 등 )
    
----
- 꼭 필요한 기준1
    - 월에 대한 데이터 기반 기준
    - 나만의 세그먼트의 RFM 지표 수렴하는 기준
    - 세그먼트 2개의 값을 최종 비교
----
- 꼭 필요한 기준2
- 마지막 분석 코호트분석
    - 202204 기준으로 VIP라는 세그먼트의 고객들은 -> 어떤 식으로 세그먼트가 월별로 이동하는가?
    - 코호트가 어떤 식으로 세그먼트가 바뀌는지에 따른 시각화 자료

## library import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import seaborn as sns
from scipy.stats import expon, poisson, gamma
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

##data

In [2]:
df=pd.read_excel('/Users/parkjunhyeok/Desktop/BDA_/지표/data/Online Retail.xlsx')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [3]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

## 데이터 파악

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
